In [20]:
import numpy as np

In [21]:
commande = {
    # switch
        # mode vol / grappin
    'state_switch_1': ..., # 0 --> position haut du switch 1 / 1 --> position basse du switch 1
    'state_switch_2': ..., # 0 --> position haut du switch 2 / 1 --> position basse du switch 2
    'state_switch_3': ..., # 0 --> position haut du switch 3 / 1 --> position milieu du switch 3 / 2 --> position basse du switch 3
    'state_switch_4': ..., # 0 --> position haut du switch 4 / 1 --> position basse du switch 4

    # position slider 
    'slider': ..., # valeur entre 0 et 100

    # position joystick
    'joystick': [..., ...], # [channel_1, channel_2]

    # potentiomètre 1
    'potentiometre_1': ..., # modification de la vitesse de rotation des moteurs grappin
}

In [22]:
def map_linear(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)


In [23]:
# retour du joystick pour direction

def GetCommMotors(ch1, ch2):

    DEAD_ZONE = 0.2    # 0 <= x <= 1

    r = abs(ch1) + abs(ch2)
    r_mapped = map_linear(r, DEAD_ZONE, 1, 0, 1)

    if r <= DEAD_ZONE:
        ch1_ = 0
        ch2_ = 0
    elif r >= DEAD_ZONE and r <= 1:
        ch1_ = ch1 * r_mapped
        ch2_ = ch2 * r_mapped
    elif r >= 1:
        ch1_ = ch1/r
        ch2_ = ch2/r

    Fv_G = -ch1_
    Fv_D = ch1_
    Fp_G = ch2_
    Fp_D = ch2_

    F_G = Fv_G + Fp_G
    F_D = Fv_D + Fp_D

    if F_G < 0:
        angle_G = 0
    elif F_G == 0:
        angle_G = 90
    elif F_G > 0:
        angle_G = 180

    if F_D < 0:
        angle_D = 0
    elif F_D == 0:
        angle_D = 90
    elif F_D > 0:
        angle_D = 180

    return abs(F_G), abs(F_D), angle_G, angle_D

In [24]:
def GetSpeedGrappin(ch3):
    
    DEAD_ZONE = 0.2     # 0 <= x <= 2
    MAX_SPEED = 1000
    
    if ch3 < -DEAD_ZONE/2:
        speedGrappin = map_linear(ch3, -1, -DEAD_ZONE/2, -MAX_SPEED, 0)

    elif (-DEAD_ZONE/2 <= ch3 and ch3 <= DEAD_ZONE/2):
        speedGrappin = 0

    elif (DEAD_ZONE/2 < ch3):
        speedGrappin = map_linear(ch3, DEAD_ZONE/2, 1, 0, MAX_SPEED)
    
    return speedGrappin

In [25]:
def FlightControl(commande) -> dict:

    speed_motor_G, speed_motor_D, angle_servomot_G, angle_servomot_D = GetCommMotors(commande['joystick'][0], commande['joystick'][1])

    instruction_motor = {

        # gestion moteur avant droit
        'speed_AvD': speed_motor_D,
        'angle_AvD': angle_servomot_D,

        # moteur avant gauche
        'speed_AvG': speed_motor_G,
        'angle_AvG': angle_servomot_G,

        # moteur arrière droit
        'speed_ArD': speed_motor_D,
        'angle_ArD': angle_servomot_D,

        # moteur arrière gauche
        'speed_ArG': speed_motor_G,
        'angle_ArG': angle_servomot_G

    }

    speed_stepmotor = GetSpeedGrappin(commande['slider']) # valeur entre -1 et 1

    instruction_grappin = {
        # control des vitesses de rotation des moteurs pas à pas
        'speed_stepmotor' : speed_stepmotor,
        'electro_aimant': commande['state_switch_1'], # 1 -> aimant actif, 0 --> aimant désactivé
    }

    control = {
        'instruction_motor': instruction_motor,
        'instruction_grappin': instruction_grappin,
    }


    return control

```python
IMU_i = {
    'a_x': ...,
    'a_y': ...,
    'a_z': ...,
    'v_p': ...,
    'v_q': ...,
    'v_r': ...
}
```

M = I * a_rot

In [66]:
def TransformIMUvalue(IMU_1: dict, IMU_2: dict, delta_t: float):

    m_struct = 3

    def InertieMomentum(a, b, m_tot):
        I = (1/5) * m_tot * (np.square(a) + np.square(b))
        return I
    
    Ix = InertieMomentum(a=0.75, b=0.75, m_tot=m_struct)
    Iy = InertieMomentum(a=3, b=0.75, m_tot=m_struct)
    Iz = InertieMomentum(a=3, b=0.75, m_tot=m_struct)

    # TODO(1): faire delta omega en valeur absolue ? 

    # expression des moments (si c'est ok)
    M_roulis = Ix * (np.deg2rad(IMU_1['v_p']) - np.deg2rad(IMU_2['v_p']))/delta_t
    M_tangage = Iy * (np.deg2rad(IMU_1['v_q']) - np.deg2rad(IMU_2['v_q']))/delta_t
    M_lacet = Iz * (np.deg2rad(IMU_1['v_r']) - np.deg2rad(IMU_2['v_r']))/delta_t

    # expression des angles d'Euler
    a_x = np.mean([[IMU_1['a_x'], IMU_2['a_x']]])
    a_y = np.mean([[IMU_1['a_y'], IMU_2['a_y']]])
    a_z = np.mean([[IMU_1['a_z'], IMU_2['a_z']]])

    roll = np.arctan(a_y/a_z)
    pitch = np.arcsin(a_x/9.80665)

    # expression des forces
    Fx = m_struct * a_x
    Fy = m_struct * a_y
    Fz = m_struct * a_z

    dynamique = {
        'momentum': {
            'M_roulis': M_roulis,
            'M_tangage': M_tangage,
            'M_lacet': M_lacet,
        },

        'orientation': {
            'roll': roll,
            'pitch': pitch,
        },

        'force': {
            'Fx': Fx,
            'Fy': Fy,
            'Fz': Fz
        },
    }

    return dynamique


In [67]:
IMU_1 = {
    'a_x': 0.05,
    'a_y': 0.0001,
    'a_z':0.078,
    'v_p': 0.02,
    'v_q': 0.035,
    'v_r': 0.004,
}

IMU_2 = {
    'a_x': 0.048,
    'a_y': 0.0003,
    'a_z':0.084,
    'v_p': 0.014,
    'v_q': 0.037,
    'v_r': 0.0045,
}

dynamique = TransformIMUvalue(IMU_1, IMU_2, 1e-4)

In [68]:
dynamique['momentum']

{'M_roulis': 0.7068583470577037,
 'M_tangage': -2.002765316663486,
 'M_lacet': -0.500691329165873}

In [69]:
dynamique['orientation']

{'roll': 0.002469130784683695, 'pitch': 0.004996630234805545}

In [70]:
dynamique['force']

{'Fx': 0.14700000000000002, 'Fy': 0.0006, 'Fz': 0.243}